In [272]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from ast import literal_eval


from experiments_helper import format_df_table, summarize_results, get_data_model, summarize_results_multi


In [404]:
def methods_naming(name):
    if "p2ce" in name:
        return "P$^2$CE"
    elif "mapocam" in name:
        return "MAPOCAM"
    elif "dice" in name:
        return "DICE"
    elif "nice" in name:
        return "NICE"
    else:
        return name
    

def methods_coloring(name):
    if "p2ce" in name:
        return "#66c2a5"
    elif "mapocam" in name:
        return "#fc8d62"
    elif "dice" in name:
        return "#80b1d3"
    elif "nice" in name:
        return "#984ea3"
    else:
        return "#ff0000"
    
def dataset_naming(name):
    if name == "german":
        return "German Credit"
    elif name == "taiwan":
        return "Taiwan"
    elif name == "adult":
        return "Adult"
    else:
        return name

In [405]:
def plot_test(results, axs):
    outliers_ = []
    method_list = results.method.unique()
    min_x, max_x, min_y, max_y = np.inf, -np.inf, np.inf, -np.inf
    for method in method_list:
        results_r = results[results.method == method]
        x_mean = results_r.time.mean()
        y_mean = results_r.costs.mean()
        outliers_.append(results_r.outlier.mean())

        min_x = min(min_x, x_mean)
        max_x = max(max_x, x_mean)
        min_y = min(min_y, y_mean)
        max_y = max(max_y, y_mean)

        x_05 = x_mean - np.quantile(results_r.time, 0.05)
        x_95 = np.quantile(results_r.time, 0.95) - x_mean
        y_05 = y_mean - np.quantile(results_r.costs, 0.05)
        y_95 = np.quantile(results_r.costs, 0.95) - y_mean


        axs[0].scatter(
            x_mean, 
            y_mean, 
            label = methods_naming(method), 
            c = methods_coloring(method),
            zorder = 100 if "p2ce" in method else 2,
        )
        axs[0].errorbar(
            [x_mean],
            [y_mean],
            xerr = [[x_05], [x_95]],
            yerr = [[y_05], [y_95]],
            label = methods_naming(method),
            c = methods_coloring(method),
            zorder = 100 if "p2ce" in method else 2,
        )

    x_pad = (max_x - min_x) * 0.1
    y_pad = (max_y - min_y) * 0.1
    #axs[0].set_xlim(min_x - x_pad, max_x + x_pad)
    #axs[0].set_ylim(min_y - y_pad, max_y + y_pad)

    handles = [
        Line2D(
            [0],
            [0],
            marker = "o",
            color = "w",
            markerfacecolor =  methods_coloring(name),
            markersize = 10,
            label = methods_naming(name),
        ) for name in method_list
    ]
    axs[0].set_xlabel("Time (s)")
    axs[0].set_ylabel("Average Continuous Dist.")


    axs[1].barh(
        [methods_naming(name) for name in method_list],
        [o * 100 for o in outliers_],
        color = [methods_coloring(name) for name in method_list],
    )
    axs[1].set_xlabel("% Outliers")

In [406]:
fig = plt.figure(figsize = (13, 5), layout = "constrained")
axs = fig.subplots(nrows = 2, ncols = 6, height_ratios=[1, 0.6])

for i, dataset_name in enumerate(["german", "taiwan", "adult"]):
    for j, model in enumerate(["lgbm", "mlp"]):
        results = []
        if model == "lgbm":
            method_list = ["p2ce_tree_abs_diff", "mapocam_tree_abs_diff", "dice", "nice"]
        else:
            method_list = ["p2ce_deep_abs_diff", "mapocam_abs_diff", "dice", "nice"]

        for method in method_list:
            try:
                results_cur = pd.read_csv(f"../results/{model}/{dataset_name}/{method}.csv")
                results_cur = summarize_results(results_cur, dataset_name)

                if "p2ce" in method:
                    method = "p2ce_abs_diff"
                if "mapocam" in method:
                    method = "mapocam_abs_diff"
                    
                results_cur["method"] = method
                results.append(results_cur)
            except:
                pass
        
        results = pd.concat(results)
        results["costs"] = results["abs_diff_costs"]
        axs_ = [axs[0][j * 3 + i], axs[1][j * 3 + i]]
        plot_test(results, axs_)

        axs[0][j * 3 + i].set_title(f"{dataset_naming(dataset_name)} - {model.upper()}")

for i in range(1, 6):
    axs[1][i].set_yticklabels([])

plt.suptitle("Comparison with 1 Objective")
plt.savefig("../figures/percentile_results.pdf", dpi = 300)

In [408]:
# plot the legend separately

fig = plt.figure()
handles = [
    Line2D(
        [0],
        [0],
        marker = "o",
        color = "w",
        markerfacecolor =  methods_coloring(name),
        markersize = 10,
        label = methods_naming(name),
    ) for name in results.method.unique()
]
handles = handles[::-1]
plt.axis("off")
plt.legend(handles=handles, ncol = len(handles), loc='upper center', bbox_to_anchor=(0.5, -0.4),)
plt.savefig("../figures/percentile_legend.pdf", dpi = 300)

## Comaparison with multiple objectives

In [246]:
dataset = "german"
method_list = ["mapocam_tree_multi", "p2ce_tree_multi", "dice_multi"]
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

In [247]:
dataset = "taiwan"
method_list = ["mapocam_tree_multi", "p2ce_tree_multi", "dice_multi"]
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

In [248]:
dataset = "adult"
method_list = ["mapocam_tree_multi", "p2ce_tree_multi", "dice_multi"]
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/lgbm/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

In [249]:
dataset = "german"
method_list = ["mapocam_multi", "p2ce_deep_multi", "dice_multi"]
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/mlp/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

In [250]:
dataset = "taiwan"
method_list = ["mapocam_multi", "p2ce_deep_multi", "dice_multi"]
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/mlp/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

In [251]:
dataset = "adult"
method_list = ["mapocam_multi", "p2ce_deep_multi", "dice_multi"]
results = []
for method in method_list:
    results_cur = pd.read_csv(f"../results/mlp/{dataset}/{method}.csv")
    results_cur = summarize_results(results_cur, dataset)
    results_cur["method"] = method
    results.append(results_cur)
results = pd.concat(results)
format_df_table(results, "method", results.columns.tolist()[:-1])

In [276]:
from pygmo import hypervolume

In [368]:
def remove_dominated_solutions(A):
    """From a list of costs, keep only the non-dominated solutions."""
    non_dominated = []
    for i, s1 in enumerate(A):
        dominated = False
        for j, s2 in enumerate(A):
            if i == j:
                continue
            if all(s2 <= s1):
                if (s1 - s2).sum() < 1e-8:
                    # are the same solution
                    continue

                dominated = True
                break
        if not dominated:
            non_dominated.append(s1)

    return non_dominated

In [392]:
def get_experiments_multi_costs(
    dataset_name,
    method_name,
    model_name,
    keep_dominated = False
):  
    """Get the costs of the experiments for a multi-objective method.
    It will return an list of numpy arrays, where each array contains the costs of the solutions for an individual.
    """
    results = pd.read_csv(f"../results/{model_name}/{dataset_name}/{method_name}.csv")
    results = summarize_results_multi(results, dataset_name)
    costs = results[["abs_diff_costs", "max_dist_costs", "n_changes"]].values
    costs = costs.tolist()
    costs = [np.array(list(zip(*c))) for c in costs]
    if not keep_dominated:
        costs = [remove_dominated_solutions(c) for c in costs]
    return costs

In [393]:
results = []
for dataset in ["german", "taiwan", "adult"]:
    costs = {}
    method_list = ["p2ce_tree_multi", "mapocam_tree_multi",  "dice_multi"]
    for method in method_list:
        costs[method] = get_experiments_multi_costs(dataset, method, "lgbm", keep_dominated = False)


    # get maximum cost for each objective
    max_costs = np.ones(len(costs[method][0][0])) * -np.inf

    for method in method_list:
        for i in range(50): # for each individual
            if len(costs[method][i]) == 0:
                continue
            max_costs = np.maximum(max_costs, np.max(costs[method][i], axis = 0))

    
    # calculate hypervolume of each method
    for method in method_list:
        for i in range(50):
            if len(costs[method][i]) == 0:
                hv = 0
            else:
                costs_ = costs[method][i]
                costs_ /= max_costs
                hv = hypervolume(costs_).compute(np.ones(len(max_costs)))

            results.append([dataset, method, i, hv])
results = pd.DataFrame(results, columns = ["dataset", "method", "individual", "hv"])

def make_table(df):
    hv_mean = df["hv"].mean().round(2)
    hv_std = df["hv"].std().round(2)
    return f"{hv_mean} ($\pm$ {hv_std})"
results.groupby(["dataset", "method"]).apply(make_table)

In [395]:
results = []
for dataset in ["german", "taiwan", "adult"]:
    costs = {}
    method_list = ["p2ce_deep_multi", "mapocam_multi",  "dice_multi"]
    for method in method_list:
        costs[method] = get_experiments_multi_costs(dataset, method, "mlp", keep_dominated = False)


    # get maximum cost for each objective
    max_costs = np.ones(len(costs[method][0][0])) * -np.inf

    for method in method_list:
        for i in range(50): # for each individual
            if len(costs[method][i]) == 0:
                continue
            max_costs = np.maximum(max_costs, np.max(costs[method][i], axis = 0))

    
    # calculate hypervolume of each method
    for method in method_list:
        for i in range(50):
            if len(costs[method][i]) == 0:
                hv = 0
            else:
                costs_ = costs[method][i]
                costs_ /= max_costs
                hv = hypervolume(costs_).compute(np.ones(len(max_costs)))

            results.append([dataset, method, i, hv])
results = pd.DataFrame(results, columns = ["dataset", "method", "individual", "hv"])

def make_table(df):
    hv_mean = df["hv"].mean().round(2)
    hv_std = df["hv"].std().round(2)
    return f"{hv_mean} ($\pm$ {hv_std})"
results.groupby(["dataset", "method"]).apply(make_table)

## Example comparison of Counterfactual Explanations

In [266]:
i = 2
dataset_name = "adult"
SEED = 0
dataset, X_train, _, model, outlier_detection, individuals = get_data_model(dataset_name, "MLPClassifier")
individuals = individuals.sample(n = 50, random_state=SEED)

# get the solution of individual i for each method
method_list = ["p2ce_deep_multi", "mapocam_multi", "dice_multi"]
df_temp = []
for method in method_list:
    results = pd.read_csv(f"../results/mlp/{dataset_name}/{method}.csv")

    solutions = literal_eval(results["solutions"].iloc[i])
    for s in solutions:
        s_df = pd.DataFrame([s], columns = X_train.columns.tolist())
        extra_columns = [
            method,
            outlier_detection.predict(np.array([s]))[0],
            model.predict_proba(s_df)[0, 1],
        ]
        df_temp.append(s + extra_columns)

extra_columns = [
    "individual",
    1,
    model.predict_proba(individuals.iloc[i:i+1])[0, 1],
]
df_temp.append(
    individuals.iloc[i].tolist() + extra_columns
)
df_temp = df_temp[::-1]
df_temp = pd.DataFrame(df_temp, columns = X_train.columns.tolist() + ["method", "outlier", "pred"])


In [267]:
not_unique_value_cols = []
for col in df_temp.columns:
    if len(df_temp[col].unique()) > 1:
        not_unique_value_cols.append(col)
df_temp = df_temp[not_unique_value_cols]

In [270]:
df_temp